    Problem Statement.

    Table: Product

    +---------------+---------+
    | Column Name   | Type    |
    +---------------+---------+
    | product_id    | int     |
    | product_name  | varchar |
    +---------------+---------+
    product_id is the primary key for this table.
    product_name is the name of the product.



    Table: Sales

    +---------------------+---------+
    | Column Name         | Type    |
    +---------------------+---------+
    | product_id          | int     |
    | period_start        | date    |
    | period_end          | date    |
    | average_daily_sales | int     |
    +---------------------+---------+
    product_id is the primary key for this table. 
    period_start and period_end indicates the start and end date for sales period, both dates are inclusive.
    The average_daily_sales column holds the average daily sales amount of the items for the period.

    Write an SQL query to report the Total sales amount of each item for each year, with corresponding product name, product_id, product_name and report_year.

    Dates of the sales years are between 2018 to 2020. Return the result table ordered by product_id and report_year.

    The query result format is in the following example:

    Product table:
    +------------+--------------+
    | product_id | product_name |
    +------------+--------------+
    | 1          | LC Phone     |
    | 2          | LC T-Shirt   |
    | 3          | LC Keychain  |
    +------------+--------------+

    Sales table:
    +------------+--------------+-------------+---------------------+
    | product_id | period_start | period_end  | average_daily_sales |
    +------------+--------------+-------------+---------------------+
    | 1          | 2019-01-25   | 2019-02-28  | 100                 |
    | 2          | 2018-12-01   | 2020-01-01  | 10                  |
    | 3          | 2019-12-01   | 2020-01-31  | 1                   |
    +------------+--------------+-------------+---------------------+

    Result table:
    +------------+--------------+-------------+--------------+
    | product_id | product_name | report_year | total_amount |
    +------------+--------------+-------------+--------------+
    | 1          | LC Phone     |    2019     | 3500         |
    | 2          | LC T-Shirt   |    2018     | 310          |
    | 2          | LC T-Shirt   |    2019     | 3650         |
    | 2          | LC T-Shirt   |    2020     | 10           |
    | 3          | LC Keychain  |    2019     | 31           |
    | 3          | LC Keychain  |    2020     | 31           |
    +------------+--------------+-------------+--------------+
    LC Phone was sold for the period of 2019-01-25 to 2019-02-28, and there are 35 days for this period. Total amount 35*100 = 3500. 
    LC T-shirt was sold for the period of 2018-12-01 to 2020-01-01, and there are 31, 365, 1 days for years 2018, 2019 and 2020 respectively.
    LC Keychain was sold for the period of 2019-12-01 to 2020-01-31, and there are 31, 31 days for years 2019 and 2020 respectively.

# Date functions

In [ ]:
WITH sales_count AS
(SELECT product_id, product_name,
CASE WHEN DATE('2018-12-31') < period_start THEN 0 
     ELSE average_daily_sales *  (1 + DATEDIFF(LEAST(DATE('2018-12-31'), period_end), 
                                               GREATEST(period_start, DATE('2018-01-01')))) END AS sales_2018,
CASE WHEN DATE('2019-12-31') < period_start THEN 0 
     ELSE average_daily_sales *  (1 + DATEDIFF(LEAST(DATE('2019-12-31'), period_end), 
                                               GREATEST(period_start, DATE('2019-01-01')))) END AS sales_2019,
CASE WHEN DATE('2020-12-31') < period_start THEN 0 
     ELSE average_daily_sales *  (1 + DATEDIFF(LEAST(DATE('2020-12-31'), period_end), 
                                               GREATEST(period_start, DATE('2020-01-01')))) END AS sales_2020
FROM Product
LEFT JOIN SALES USING(product_id)),

temp as
(SELECT CAST(product_id as CHAR(5)) AS product_id, product_name, '2018' as report_year, SUM(sales_2018) AS total_amount 
FROM sales_count
WHERE sales_2018 > 0
GROUP BY 1,2,3

UNION 

SELECT  CAST(product_id as CHAR(5)) AS product_id, product_name, '2019' as report_year, SUM(sales_2019) AS total_amount
FROM sales_count
WHERE sales_2019 > 0
GROUP BY 1,2,3
 
UNION

SELECT  CAST(product_id as CHAR(5)) AS product_id, product_name, '2020' as report_year, SUM(sales_2020) AS total_amount
FROM sales_count
WHERE sales_2020 > 0
GROUP BY 1,2,3)

SELECT * from temp
ORDER BY 1, 3

# Shorter using Date Functions

In [ ]:
SELECT a.product_id, b.product_name, a.report_year, a.total_amount
FROM (
    SELECT product_id, '2018' AS report_year,
        average_daily_sales * (DATEDIFF(LEAST(period_end, '2018-12-31'), GREATEST(period_start, '2018-01-01'))+1) AS total_amount
    FROM Sales
    WHERE YEAR(period_start)=2018 OR YEAR(period_end)=2018

    UNION ALL

    SELECT product_id, '2019' AS report_year,
        average_daily_sales * (DATEDIFF(LEAST(period_end, '2019-12-31'), GREATEST(period_start, '2019-01-01'))+1) AS total_amount
    FROM Sales
    WHERE YEAR(period_start)<=2019 AND YEAR(period_end)>=2019

    UNION ALL

    SELECT product_id, '2020' AS report_year,
        average_daily_sales * (DATEDIFF(LEAST(period_end, '2020-12-31'), GREATEST(period_start, '2020-01-01'))+1) AS total_amount
    FROM Sales
    WHERE YEAR(period_start)=2020 OR YEAR(period_end)=2020
) a
LEFT JOIN Product b
ON a.product_id = b.product_id
ORDER BY a.product_id, a.report_year

# Recursive With

In [ ]:
WITH RECURSIVE CTE AS
    (SELECT MIN(period_start) as date
     FROM Sales 
     UNION ALL
     SELECT DATE_ADD(date, INTERVAL 1 day)
     FROM CTE
     WHERE date <= ALL (SELECT MAX(period_end) FROM Sales))

 
SELECT 
s.product_id, p.product_name, LEFT(e.date,4) as report_year, SUM(s.average_daily_sales) as total_amount
FROM Sales s
JOIN Product p ON p.product_id = s.product_id
JOIN CTE e ON s.period_start<=e.date AND s.period_end>=e.date
GROUP BY 1,2,3 
ORDER BY 1,3